In [1]:
#setup
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
sns.set_theme()
plt.rcParams["figure.figsize"] = (10,8)
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
from main import nlp_pipeline_result

In [ ]:
#data_ner = nlp_pipeline_result(disable_ner = False)
#data.to_csv('data/data_ner.csv')

data_non_ner = nlp_pipeline_result()
data.to_csv('data/data.csv')

In [9]:
data.to_csv('data/data_ner.csv')

In [8]:
for i in range(0,50,10):
  display(data[i:i+10])

,Original Tweets,Parsed Tweets,Author,Party,Lemmas
0,El Centro de Robótica e Inteligencia Artificia...,El Centro de Robótica e Inteligencia Artificia...,José Luis Ábalos Meco,PSOE,Centro Robótica Inteligencia Artificial Alcáza...
1,Día de homenaje a los españoles deportados y f...,Día de homenaje a los españoles deportados y f...,José Luis Ábalos Meco,PSOE,homenaje español deportado fallecido campo con...
2,Mi agradecimiento a los interventores y apoder...,Mi agradecimiento a los interventores y apoder...,José Luis Ábalos Meco,PSOE,agradecimiento interventor apoderado represent...
3,Hemos autorizado las obras de un puente móvil ...,Hemos autorizado las obras de un puente móvil ...,José Luis Ábalos Meco,PSOE,autorizar obra puente móvil conexión ferroviar...
4,La estación de Santa Justa cumple 30 años con ...,La estación de Santa Justa cumple 30 años con ...,José Luis Ábalos Meco,PSOE,estación Santa Justa cumplir año millón usuari...
5,Se cumplen 142 años de la fundación del @PSOE ...,Se cumplen 142 años de la fundación del PSOE y...,José Luis Ábalos Meco,PSOE,cumplir año fundación PSOE seguir hablar timbr...
6,La llama de nuestro fundador Pablo Iglesias Po...,La llama de nuestro fundador Pablo Iglesias Po...,José Luis Ábalos Meco,PSOE,llamar fundador Pablo Iglesias Posse vivo año ...
7,Junto con los sindicatos @UGT_Comunica y @CCOO...,Junto con los sindicatos y compañeras y compañ...,José Luis Ábalos Meco,PSOE,sindicato compañera compañera clase trabajador...
8,Mi homenaje a los trabajadores y trabajadoras ...,Mi homenaje a los trabajadores y trabajadoras ...,José Luis Ábalos Meco,PSOE,homenaje trabajador trabajador respuesta pande...
9,"Reunido con el Secretario de @SRE_mx, @m_ebrar...",Reunido con el Secretario de para abordar los ...,José Luis Ábalos Meco,PSOE,reunido Secretario abordar proyecto Tren Maya ...


,Original Tweets,Parsed Tweets,Author,Party,Lemmas
10,La Junta Electoral abre dos expedientes sancio...,La Junta Electoral abre dos expedientes sancio...,José Luis Ábalos Meco,PSOE,Junta Electoral abrir expediente sancionador A...
11,La Justicia recuerda a Vox que una querella ne...,La Justicia recuerda a Vox que una querella ne...,José Luis Ábalos Meco,PSOE,Justicia recordar Vox querella necesitar razón...
12,Una vez más se demuestra que la arquitectura e...,Una vez más se demuestra que la arquitectura e...,José Luis Ábalos Meco,PSOE,demostrar arquitectura español alto nivel cont...
13,Damos nuevos pasos para restablecer el tráfico...,Damos nuevos pasos para restablecer el tráfico...,José Luis Ábalos Meco,PSOE,dar paso restablecer tráfico ferroviario Almer...
14,Vamos a poner en marcha en #Linares un nuevo C...,Vamos a poner en marcha en un nuevo Centro de ...,José Luis Ábalos Meco,PSOE,marcha Centro Competencias Digitales crear pue...
15,"Mi agradecimiento al alcalde @JulioMillanjaen,...",Mi agradecimiento al alcalde por la acogida en...,José Luis Ábalos Meco,PSOE,agradecimiento alcalde acogida confirmar compr...
16,Seguimos apostando por la recuperación de nues...,Seguimos apostando por la recuperación de nues...,José Luis Ábalos Meco,PSOE,seguir apostar recuperación patrimonio históri...
17,Le he trasladado al Secretario de Transportes ...,Le he trasladado al Secretario de Transportes ...,José Luis Ábalos Meco,PSOE,trasladar Secretario Transportes Unidos dispos...
18,"Todo mi apoyo y solidaridad, @MarotoReyes. En ...","Todo mi apoyo y solidaridad, En un país democr...",José Luis Ábalos Meco,PSOE,apoyo solidaridad país democrático imposible a...
19,"Alejandro Morant: ""¿Alguien se cree que llegue...","Alejandro Morant: ""¿Alguien se cree que llegue...",José Luis Ábalos Meco,PSOE,Alejandro Morant Alguien creer llegar ministro...


,Original Tweets,Parsed Tweets,Author,Party,Lemmas
20,"25 de abril, Día de les @cortsval. Una fecha l...","25 de abril, Día de les Una fecha llena de ref...",José Luis Ábalos Meco,PSOE,abril fecha lleno referencia histórico socieda...
21,"25 de abril de 1974, la Revolución de los Clav...","25 de abril de 1974, la Revolución de los Clav...",José Luis Ábalos Meco,PSOE,abril Revolución Claveles oir él Grândola Vila...
22,Seguimos cumpliendo con los compromisos para e...,Seguimos cumpliendo con los compromisos para e...,José Luis Ábalos Meco,PSOE,seguir cumplir compromiso desarrollo Corredor ...
23,‘El porqué de las palabras’(Francisco Brines. ...,‘El porqué de las palabras’(Francisco Brines. ...,José Luis Ábalos Meco,PSOE,porqué Brines Premio Cervantes poeta
24,Los demócratas creemos en la palabra pero no p...,Los demócratas creemos en la palabra pero no p...,José Luis Ábalos Meco,PSOE,demócrata creer palabra normalizar ultraderech...
25,Este Gobierno invierte más en Extremadura. Hem...,Este Gobierno invierte más en Extremadura. Hem...,José Luis Ábalos Meco,PSOE,Gobierno invertir Extremadura licitar inversió...
26,Este verano ponemos en marcha en #Mérida un Ce...,Este verano ponemos en marcha en un Centro Tec...,José Luis Ábalos Meco,PSOE,verano poner marcha Centro Tecnológico Avanzad...
27,Tenemos que aislar a quienes siembran el odio ...,Tenemos que aislar a quienes siembran el odio ...,José Luis Ábalos Meco,PSOE,aislar sembrar odio señalar bala demócrata sol...
28,Con el proyecto de Acondicionamiento de la Ave...,Con el proyecto de Acondicionamiento de la Ave...,José Luis Ábalos Meco,PSOE,proyecto Acondicionamiento Avenida Martin Cara...
29,"Cumpliendo nuestros compromisos, hemos multipl...","Cumpliendo nuestros compromisos, hemos multipl...",José Luis Ábalos Meco,PSOE,cumplir compromiso multiplicar licitación inve...


,Original Tweets,Parsed Tweets,Author,Party,Lemmas
30,Visitando la nueva Base de Mantenimiento de Re...,Visitando la nueva Base de Mantenimiento de Re...,José Luis Ábalos Meco,PSOE,visitar Base Mantenimiento Renfe Badajoz acoge...
31,"Hoy, he comprobado personalmente en Badajoz qu...","Hoy, he comprobado personalmente en Badajoz qu...",José Luis Ábalos Meco,PSOE,comprobar Badajoz ejecutado trabajo remodelaci...
32,⚪ Licitaremos antes del verano las obras de la...,⚪ Licitaremos antes del verano las obras de la...,José Luis Ábalos Meco,PSOE,Licitaremos verano obra variante Zafra Badajoz...
33,Seguimos recuperando nuestro patrimonio histór...,Seguimos recuperando nuestro patrimonio histór...,José Luis Ábalos Meco,PSOE,seguir recuperar patrimonio histórico reabrimo...
34,Damos un nuevo paso en la Alta Velocidad Madri...,Damos un nuevo paso en la Alta Velocidad Madri...,José Luis Ábalos Meco,PSOE,dar paso Alta Velocidad iniciar prueba sistema...
35,Reunido con el ministro marroquí de Infraestru...,Reunido con el ministro marroquí de Infraestru...,José Luis Ábalos Meco,PSOE,reunido ministro marroquí Infraestructura Tran...
36,🟣 @Renfe sigue avanzando en la implantación de...,🟣 sigue avanzando en la implantación de sus Ce...,José Luis Ábalos Meco,PSOE,avanzar implantación Centros Competencias Digi...
37,Cumplimos nuestro objetivo de financiar el tra...,Cumplimos nuestro objetivo de financiar el tra...,José Luis Ábalos Meco,PSOE,cumplir objetivo financiar transporte público ...
38,Esta tarde he acudido a la cita de @GVAsanitat...,Esta tarde he acudido a la cita de para recibi...,José Luis Ábalos Meco,PSOE,acudir cita recibir vacuna AstraZeneca Valenci...
39,Mi saludo y reconocimiento a @facua que hoy ce...,Mi saludo y reconocimiento a que hoy celebra s...,José Luis Ábalos Meco,PSOE,saludo reconocimiento celebrar Asamblea agrade...


,Original Tweets,Parsed Tweets,Author,Party,Lemmas
40,Vamos a potenciar el tráfico ferroviario de me...,Vamos a potenciar el tráfico ferroviario de me...,José Luis Ábalos Meco,PSOE,potenciar tráfico ferroviario mercancía Plan M...
41,Iniciamos una nueva etapa en la historia de la...,Iniciamos una nueva etapa en la historia de la...,José Luis Ábalos Meco,PSOE,iniciar etapa historia estación Canfranc puest...
42,He recibido de mis compañeros y compañeras del...,He recibido de mis compañeros y compañeras del...,José Luis Ábalos Meco,PSOE,recibir compañero compañera enorme orgullo Pre...
43,Estamos comprometidos con el derecho a la #Viv...,Estamos comprometidos con el derecho a la Por ...,José Luis Ábalos Meco,PSOE,comprometido derecho ampliar mes carácter urge...
44,Los casi 7.000 Mill€ para la rehabilitación de...,Los casi 7.000 Mill€ para la rehabilitación de...,José Luis Ábalos Meco,PSOE,rehabilitación edificio regeneración Plan Recu...
45,El volumen de inversiones de @mitmagob incluid...,El volumen de inversiones de incluido en los e...,José Luis Ábalos Meco,PSOE,volumen inversión incluido contribuir año crec...
46,Avanzamos en la modernización y mejora de @rod...,Avanzamos en la modernización y mejora de el C...,José Luis Ábalos Meco,PSOE,avanzar modernización mejora Consejo Ministros...
47,Hemos autorizado las obras de mejora de la Est...,Hemos autorizado las obras de mejora de la Est...,José Luis Ábalos Meco,PSOE,autorizar obra mejora Estación Marítima cercan...
48,Seguimos trabajando en la primera de Ley de Vi...,Seguimos trabajando en la primera de Ley de Vi...,José Luis Ábalos Meco,PSOE,seguir trabajar Ley Vivienda país frenar subid...
49,Todo mi reconocimiento a los hombres y mujeres...,Todo mi reconocimiento a los hombres y mujeres...,José Luis Ábalos Meco,PSOE,reconocimiento hombre mujer integrar contribuc...


In [ ]:
data['n_lemmas'] = 0

In [ ]:
data = pd.DataFrame({'tweets_text': raw_data[0], 
                     'tweets_lemmas': raw_data.index})

In [ ]:
len(data)

In [ ]:
data = data.reset_index()
#data['n_lemmas'] = data.apply(lam)
data

In [ ]:
14929*0.2

In [ ]:
data.head()

In [ ]:
raw_data